In [1]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
)
from langchain.prompts import load_prompt, ChatPromptTemplate
from langchain.schema import AIMessage, HumanMessage, BaseMessage

from langchain_core.callbacks import BaseCallbackHandler

In [2]:
load_dotenv()

True

In [125]:
API_URL = "http://localhost:8000"

In [3]:
def unpack_messages(messages):
    unpacked = ""
    for message in messages:
        if isinstance(message, HumanMessage):
            unpacked += f"User: {message.content}\n"
        elif isinstance(message, AIMessage):
            unpacked += f"AI: {message.content}\n"
        # Add more conditions here if you're using other message types
    return unpacked


In [4]:
# llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=1.2, model_kwargs={"top_p": 0.5})
# llm = ChatOpenAI(model_name = "gpt-4", temperature=1.2, model_kwargs={"top_p": 0.5})
llm = ChatOpenAI(model_name = "gpt-4", temperature=1.2)

SYSTEM_THOUGHT = load_prompt('prompts/thought.yaml')
SYSTEM_RESPONSE = load_prompt('prompts/response.yaml')
SYSTEM_USER_PREDICTION_THOUGHT = load_prompt('prompts/user_prediction_thought.yaml')

system_thought = SystemMessagePromptTemplate(prompt=SYSTEM_THOUGHT)
system_response = SystemMessagePromptTemplate(prompt=SYSTEM_RESPONSE)
system_user_prediction_thought = SystemMessagePromptTemplate(prompt=SYSTEM_USER_PREDICTION_THOUGHT)

/Users/13point5/Library/Caches/pypoetry/virtualenvs/course-lm-backend-OOVGKqfC-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
messages = {
	"thoughts": [],
	"responses": [],
	"user_prediction_thought": []
}

Wonderings:
- Why does the `think` step only consider the previous `thoughts` and not `responses`?

In [6]:
def think(query: str):
	"""Generate Bloom's thought on the user."""

	thought_prompt = ChatPromptTemplate.from_messages([
		system_thought,
		*messages["thoughts"],
		HumanMessage(content=query)
	])

	chain = thought_prompt | llm 


	class SaveMessagesHandler(BaseCallbackHandler):
		def on_llm_end(self, response, **kwargs):
			ai_response = response.generations[0][0].text

			messages["thoughts"].append(HumanMessage(content=query))
			messages["thoughts"].append(AIMessage(content=ai_response))

	save_messages_handler = SaveMessagesHandler()

	return chain.invoke({}, config={
		"callbacks": [save_messages_handler]
	})

In [7]:
def respond(thought: str, query: str):
	"""Generate Bloom's response to the user."""

	response_prompt = ChatPromptTemplate.from_messages([
		system_response,
		*messages["responses"],
		HumanMessage(content=query)
	])

	chain = response_prompt | llm 


	class SaveMessagesHandler(BaseCallbackHandler):
		def on_llm_end(self, response, **kwargs):
			ai_response = response.generations[0][0].text

			messages["responses"].append(HumanMessage(content=query))
			messages["responses"].append(AIMessage(content=ai_response))

	save_messages_handler = SaveMessagesHandler()

	return chain.invoke({"thought": thought}, config={
		"callbacks": [save_messages_handler]
	})

Wonderings:
- Where is `user_prediction_thought` used??

In [8]:
def think_user_prediction():
	"""Generate a thought about what the user is going to say"""

	prompt = ChatPromptTemplate.from_messages([
		system_user_prediction_thought,
	])

	chain = prompt | llm

	history = unpack_messages(messages["responses"])

	user_prediction_thought = chain.invoke({"history": history})

	messages["user_prediction_thought"].append(user_prediction_thought)


In [9]:
def chat(query: str ) -> tuple[str, str]:
	thought = think(query=query)

	response = respond(thought=thought, query=query)

	think_user_prediction()

	return thought, response


In [10]:
chat("I'm not able to understand the difference between constructionism and constructivism")

(AIMessage(content="Predicted User Need: \n\nThe user is looking for a clear explanation that distinguishes between the terms constructionism and constructivism.\n\nEnhanced Prediction with Additional Data:\n\nAdditional relevant data, such as the user's knowledge level or background in learning theories, whether this request relates to their work, the reason the query is raised (a study assignment, curiosity or clarifying a discussion point), can enhance the way I frame and tailor the fitting information.\n\nClarifying Details: \n\nThe theory they comprehend better among the two could influence how the other is dissected in comparison. If its part of their job, a more comprehensive analysis could be deemed useful. Meanwhile, descriptive explanations for casual learners, and scholastic sources/citations will be highly effective for students preparing for a paper/assignment.", response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 84, 'total_tokens': 241}, 'model_

In [11]:
messages

{'thoughts': [HumanMessage(content="I'm not able to understand the difference between constructionism and constructivism"),
  AIMessage(content="Predicted User Need: \n\nThe user is looking for a clear explanation that distinguishes between the terms constructionism and constructivism.\n\nEnhanced Prediction with Additional Data:\n\nAdditional relevant data, such as the user's knowledge level or background in learning theories, whether this request relates to their work, the reason the query is raised (a study assignment, curiosity or clarifying a discussion point), can enhance the way I frame and tailor the fitting information.\n\nClarifying Details: \n\nThe theory they comprehend better among the two could influence how the other is dissected in comparison. If its part of their job, a more comprehensive analysis could be deemed useful. Meanwhile, descriptive explanations for casual learners, and scholastic sources/citations will be highly effective for students preparing for a paper/

In [12]:
print(messages['thoughts'][1].content)

Predicted User Need: 

The user is looking for a clear explanation that distinguishes between the terms constructionism and constructivism.

Enhanced Prediction with Additional Data:

Additional relevant data, such as the user's knowledge level or background in learning theories, whether this request relates to their work, the reason the query is raised (a study assignment, curiosity or clarifying a discussion point), can enhance the way I frame and tailor the fitting information.

Clarifying Details: 

The theory they comprehend better among the two could influence how the other is dissected in comparison. If its part of their job, a more comprehensive analysis could be deemed useful. Meanwhile, descriptive explanations for casual learners, and scholastic sources/citations will be highly effective for students preparing for a paper/assignment.
